In [1]:
import torch
import math
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import pandas as pd
from sklearn.model_selection import train_test_split 

In [2]:
import sys, time

def print_(*values, sep=' ', end='\n', file=sys.stdout, flush=True):
    file.write(sep.join(map(str, values)) + end)
    if flush:
        file.flush()
    
def cls_line():
    print_('', end='\r')


print_('Loading model...', end='')
time.sleep(1)
cls_line()
print_('Loading model....')

Loading model....


In [3]:
vocab_size = 100
n_embed = 64
max_tokens = 10

In [4]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000, device = torch.device('cuda')):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2)
                             * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [5]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0.1, num_heads=8, device=torch.device('cuda')):
        super(Encoder, self).__init__()
        
        self.hidden_size = hidden_size
        self.input_size = input_size
        self._dropout = dropout
        self._num_heads = num_heads
        self.device = device

        self.attention = nn.MultiheadAttention(hidden_size, num_heads, dropout=dropout, device=device)
        self.ffwd = nn.Sequential(
            nn.Linear(hidden_size, hidden_size, device=device),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size, device=device),
        )
        self.add_norm = nn.Sequential(
            nn.LayerNorm(hidden_size, device=device),
            nn.ReLU()
        )
        self.add_norm2 = nn.Sequential(
            nn.LayerNorm(hidden_size, device=device),
            nn.ReLU()
        )

    def forward(self, x):
        res, _ = self.attention(x, x, x) 
        x = self.add_norm1(res + x)
        res = self.ffwd(x)
        x = self.add_norm2(res + x)
        return x

In [6]:
class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0.1, num_heads=8, device=torch.device('cuda')):
        super(Decoder, self).__init__()

        self._hidden_size = hidden_size
        self._input_size = input_size
        self._dropout = dropout
        self._num_heads = num_heads
        self._device = device

        self.masked_attention = nn.MultiheadAttention(
            hidden_size, num_heads, dropout=dropout, device=device)
        self.enc_dec_attention = nn.MultiheadAttention(
            hidden_size, num_heads, dropout=dropout, device=device)
        self.ffwd = nn.Sequential(
            nn.Linear(hidden_size, hidden_size, device=device),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size, device=device)
        )
        self.add_norm1 = nn.Sequential(
            nn.LayerNorm(hidden_size, device=device),
            nn.ReLU()
        )
        self.add_norm2 = nn.Sequential(
            nn.LayerNorm(hidden_size, device=device),
            nn.ReLU()
        )
        self.add_norm3 = nn.Sequential(
            nn.LayerNorm(hidden_size, device=device),
            nn.ReLU()
        )

    def forward(self, x, hidden_state):
        res, _ = self.masked_attention.forward(x, x, x, 
                                               attn_mask=torch.tril(
                                                   torch.eye(self._hidden_size, 
                                                             self._hidden_size, 
                                                             dtype=torch.bool,
                                                             device=self._device))) #! test mask
        x = self.add_norm1(res + x)
        res, _ = self.enc_dec_attention.forward(x, hidden_state, hidden_state) #! check if its right
        x = self.add_norm2(res + x)
        res = self.ffwd(x)
        x = self.add_norm3(res + x)
        return x

In [7]:
class EncoderBlock(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0.1, num_heads=8, n_encoders=6, device=torch.device('cuda')):
        super(EncoderBlock, self).__init__()
        
        self._hidden_size = hidden_size
        self._input_size = input_size
        self._dropout = dropout
        self._num_heads = num_heads
        self._n_encoders = n_encoders
        self._device = device

        self.embedding = nn.Embedding(input_size, hidden_size, device=device)
        self.pos_encoder = PositionalEncoding(hidden_size, device=device)

        self._encoders = [Encoder(input_size, hidden_size, dropout, num_heads, device) for _ in range(n_encoders)] 

    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_encoder(x)
        
        for encoder in self._encoders:
            x = encoder(x)
            
        return x

In [8]:
class DecoderBlock(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0.1, num_heads=8, n_decoders=6, device=torch.device('cuda')):
        super(DecoderBlock, self).__init__()

        self._hidden_size = hidden_size
        self._input_size = input_size
        self._dropout = dropout
        self._num_heads = num_heads
        self._n_decoders = n_decoders
        self._device = device

        self.embedding = nn.Embedding(input_size, hidden_size, device=device)
        self.pos_encoder = PositionalEncoding(hidden_size, device=device)
        
        self._decoders = [Decoder(
            input_size, hidden_size, dropout, num_heads, device=device) for _ in range(n_decoders)]

    def forward(self, x, hidden_state):
        x = self.embedding(x)
        x = self.pos_encoder(x)

        for decoder in self._decoders:
            x = decoder(x, hidden_state)

        return x

In [9]:
class LinearAndSoftmax(nn.Module):
    def __init__(self, hidden_size, output_size, device) -> None:
        super().__init__()
        self.linear = nn.Linear(hidden_size, output_size, device=device)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.linear(x)
        x = self.softmax(x)
        return x

In [10]:
class SimpleTransformer(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.1, num_heads=8, n_encoders=6, n_decoders=6, device=torch.device('cuda')):
        super(SimpleTransformer, self).__init__()

        self._hidden_size = hidden_size
        self._input_size = input_size
        self._output_size = output_size
        self._dropout = dropout
        self._num_heads = num_heads
        self._n_encoders = n_encoders
        self._n_decoders = n_decoders
        self._device = device

        self.encoder = EncoderBlock(input_size, hidden_size, dropout, num_heads, n_encoders, device)
        self.decoder = DecoderBlock(input_size, hidden_size, dropout, num_heads, n_decoders, device)
        self.softmax = LinearAndSoftmax(hidden_size, output_size, device)

    def forward(self, x):
        hidden_state = self.encoder(x)
        x = self.decoder(x, hidden_state)
        x = self.softmax(x)
        return x


In [11]:
LinearAndSoftmax(10, 10, torch.device('cpu'))(torch.rand(10, 10))

tensor([[0.0841, 0.1102, 0.1152, 0.1208, 0.0839, 0.1051, 0.0823, 0.1548, 0.0514,
         0.0921],
        [0.0715, 0.0967, 0.1073, 0.1269, 0.1178, 0.0747, 0.0877, 0.1757, 0.0440,
         0.0977],
        [0.0695, 0.1291, 0.1358, 0.1166, 0.1325, 0.0691, 0.0817, 0.1387, 0.0424,
         0.0846],
        [0.0903, 0.1199, 0.1258, 0.1396, 0.1120, 0.0635, 0.0708, 0.1117, 0.0534,
         0.1130],
        [0.0967, 0.1528, 0.1363, 0.1225, 0.1032, 0.0578, 0.0781, 0.1009, 0.0548,
         0.0969],
        [0.0870, 0.0853, 0.1185, 0.1565, 0.0836, 0.1219, 0.0633, 0.1179, 0.0463,
         0.1196],
        [0.0710, 0.1182, 0.1200, 0.1172, 0.1209, 0.0891, 0.0701, 0.1410, 0.0498,
         0.1027],
        [0.0892, 0.1397, 0.1138, 0.1065, 0.1070, 0.0759, 0.0661, 0.1204, 0.0712,
         0.1103],
        [0.0957, 0.1541, 0.0917, 0.0814, 0.1357, 0.0667, 0.0666, 0.1508, 0.0450,
         0.1124],
        [0.0971, 0.1033, 0.1058, 0.1186, 0.0935, 0.0887, 0.0692, 0.1670, 0.0522,
         0.1045]], grad_fn=<

In [12]:
from dataclasses import dataclass
from typing import Callable


@dataclass(frozen=True, slots=True)
class DataProcessor:

    processor: Callable

    def __call__(self, *args, **kwargs):
        return self.processor(*args, **kwargs)
    
    
@dataclass(frozen=True, slots=True)
class Model:
    preprocessor: DataProcessor
    ai: nn.Module
    postprocessor: DataProcessor

    def predict(self, *args, **kwargs):
        return self.postprocessor(self.ai(self.preprocessor(*args, **kwargs)))

    def predict_no_postprocessing(self, *args, **kwargs):
        return self.ai(self.preprocessor(*args, **kwargs))

    def __call__(self, *args, **kwargs):
        return self.predict(*args, **kwargs) if not self.ai.training else self.predict_no_postprocessing(*args, **kwargs)

In [13]:
class Trainer:

    @property
    def epochs(self): return self._epochs

    @epochs.setter
    def epochs(self, epochs):
        if epochs <= 0:
            raise ValueError('Number of epochs must be greater than zero')
        elif not isinstance(epochs, int):
            raise TypeError('Number of epochs must be an integer')

        self._epochs = epochs

    @property
    def batch_size(self): return self._batch_size

    @batch_size.setter
    def batch_size(self, batch_size):
        if batch_size <= 0:
            raise ValueError('Batch size must be greater than zero')
        elif not isinstance(batch_size, int):
            raise TypeError('Batch size must be an integer')

        self._batch_size = batch_size

    @property
    def model(self): return self._model

    @model.setter
    def model(self, model):
        if not isinstance(model, Model):
            raise TypeError('Model must be a Model object')
        self._model = model

    @property
    def X_dataset(self): return self._X_dataset

    @X_dataset.setter
    def X_dataset(self, X_dataset):
        self._X_dataset = X_dataset

    @property
    def y_dataset(self): return self._y_dataset

    @y_dataset.setter
    def y_dataset(self, y_dataset):
        self._y_dataset = y_dataset

    @property
    def test_split(self): return self._test_split

    @test_split.setter
    def test_split(self, test_split):
        if not isinstance(test_split, float):
            raise TypeError('Test split must be a float')
        elif test_split <= 0 or test_split >= 1:
            raise ValueError('Test split must be between 0 and 1')
        self._test_split = test_split

    @property
    def valid_split(self): return self._valid_split

    @valid_split.setter
    def valid_split(self, valid_split):
        if not isinstance(valid_split, float):
            raise TypeError('Validation split must be a float')
        elif valid_split <= 0 or valid_split >= 1:
            raise ValueError('Validation split must be between 0 and 1')
        self._valid_split = valid_split

    def __init__(self, model: Model,
                 epochs: int = 100,
                 batch_size: int = 64,
                 X_dataset=None,
                 y_dataset=None,
                 test_split: float = 0.2,
                 valid_split: float = 0.2,
                 gen_test: int | None = None,
                 gen_valid: int | None = None,
                 device: torch.device = torch.device('cuda')) -> None:

        self.epochs = epochs
        self.batch_size = batch_size
        self.model = model
        self.X_dataset = X_dataset
        self.y_dataset = y_dataset
        self.test_split = test_split
        self.valid_split = valid_split
        self.gen_test = gen_test
        self.gen_valid = gen_valid
        self._device = device

    def _split_dataset(self):
        Xtrain, self._X_test, ytrain, self._y_test = train_test_split(
            self.X_dataset, self.y_dataset, test_size=self.test_split, random_state=self.gen_test)
        self._X_train, self._X_valid, self._y_train, self._y_valid = train_test_split(
            Xtrain, ytrain, test_size=self.valid_split, random_state=self.gen_valid)

    split_data = property(lambda self: {
        'training': (self._X_train, self._y_train),
        'validation': (self._X_valid, self._y_valid),
        'test': (self._X_test, self._y_test)
    } if hasattr(self, '_X_train') else None)

    def fit(self, optimizer, loss_function):
        self._split_dataset()

        train_loader = data.DataLoader(data.TensorDataset(
            self._X_train, self._y_train), batch_size=self.batch_size, shuffle=True)
        validation_dataloader = data.DataLoader(data.TensorDataset(
            self._X_valid, self._y_valid), batch_size=self.batch_size, shuffle=True)

        for epoch in range(self.epochs):
            for X, y in train_loader:
                X, y = X.to(self._device), y.to(self._device)
                self.model.ai.train()
                optimizer.zero_grad()
                y_pred = self.model(X)
                loss = loss_function(y_pred, y)
                loss.backward()
                optimizer.step()
            self.model.ai.eval()
            val_loss = 0
            with torch.no_grad():
                for val_inputs, val_targets in validation_dataloader:
                    val_inputs, val_targets = val_inputs.to(
                        self._device), val_targets.to(self._device)
                    val_outputs = self.model.predict_no_postprocessing(
                        val_inputs)
                    val_loss += loss_function(val_outputs, val_targets).item()

            val_loss /= len(validation_dataloader)
            print(
                f'Epoch {epoch + 1}/{self.epochs} | Validation Loss: {val_loss}, Loss: {loss.item()}')

    def report(self):
        pass

    def graphics(self):
        pass